In [2]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

from selenium import  webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.alert import Alert

In [3]:
#크롤링 + 화면띄우기
def ssg_shopping_list(order_list):
    
    ssg_list = []
    base_url = 'http://earlymorning.ssg.com/search.ssg'
    driver = webdriver.Chrome()
    
    for word in order_list[:-1]:
        

        params = {'target':'all','query':word}
        resp = requests.get(base_url, params=params)
        soup = BeautifulSoup(resp.text,'lxml')


        ul_tag = soup.find('ul', class_="cunit_thmb_lst")
        items = ul_tag.find_all('li', class_='cunit_t232')

        no_item = {
            '상품명':word,
            '원가격':" ",
            '할인율':" ",
            '최종가격':" ",
            '비고':'해당상품이 없습니다.'}
        


        if items != []:
            item_list = []   
            for item in items[:6]:

                item_info = item.find('div',class_="cunit_info")

                #이름        
                if item_info.find('strong', class_='brd') == None:
                    item_name = item_info.find('em', class_='tx_ko').text
                else:
                    item_name = item_info.find_all('em', class_='tx_ko')[0].text +" "+ item_info.find_all('em', class_='tx_ko')[1].text


                if item.find('div',class_="soldout_area") == None:
                    #할인가격
                    item_price = item_info.find('em', class_='ssg_price').text

                    #원가격
                    item_org_price = item_info.find('div',class_ = 'org_price')
                    if item_org_price ==  None:
                        item_org_price = item_price
                    else:
                        item_org_price = item_info.find('div',class_ = 'org_price').find('em', class_='ssg_price').text

                    #할인율(에누리, 쿠폰)
                    item_sale_percent = item.find('span', class_='di_enuri')
                    if item_sale_percent == None:

                        item_sale_percent = item.find('span', class_='di_coupon')
                        if item_sale_percent == None :
                            item_sale_percent = " "
                        else:
                            item_sale_percent = item.find('span', class_='di_coupon').text

                    else:
                        item_sale_percent = item.find('span', class_='di_enuri').text



                    #비고
                    item_plus = item.find('span', class_='di_plus')
                    if item_plus == None:
                        item_plus = " "
                    else:
                        item_plus = item.find('span', class_='di_plus').text


                    #item_list에 넣기
                    item_list.append({
                        '상품명':item_name,
                        '원가격':item_org_price,
                        '할인율':item_sale_percent,
                        '최종가격':item_price,
                        '비고':item_plus
                     })
                else:

                    soldout_item = {
                        '상품명':item_name,
                        '원가격':"0",
                        '할인율':"0",
                        '최종가격':"0",
                        '비고':'해당상품은 품절입니다.'                   
                    }
                    item_list.append(soldout_item)



            best_item = item_list[0]
            
            #장바구니담기
            cart_url = 'http://earlymorning.ssg.com/search.ssg?target=all&query='
   
            driver.get(cart_url+word)

            thmb = driver.find_element_by_class_name('thmb')
            util_bx = driver.find_elements_by_class_name('util_bx')
            cart = util_bx[0].find_elements_by_tag_name('a')[2]

                # action
            actions = ActionChains(driver)

            actions.move_to_element(thmb)
            actions.perform()
            actions.reset_actions()

            cart = util_bx[0].find_elements_by_tag_name('a')[2]
            actions.click(cart)
            actions.perform()

            Alert(driver).accept()
            actions.reset_actions()


        else:
            best_item = no_item
    
        ssg_list.append(best_item)
    
    #마지막장바구니화면 
    util_menu = driver.find_element_by_class_name('util_menu')
    final_cart = util_menu.find_elements_by_tag_name('a')[1]
    final_cart.click()
                    
    return ssg_list
            
    

In [4]:
# 데이터프레임 + 엑셀저장
def shoppinglist_ssg(thelist):
    data_ssg = pd.DataFrame(thelist)
    
    data_ssg["원가격"] = data_ssg["원가격"].str.replace(",",'').astype(int)
    data_ssg["최종가격"] = data_ssg["최종가격"].str.replace(",",'').astype(int)
    
    org_total = data_ssg["원가격"].sum()
    total = data_ssg["최종가격"].sum()
    
    data_ssg = data_ssg.append(
        {'상품명':"**합계**", "원가격":org_total, '할인율':total - org_total, '최종가격':total, '비고':""},
            ignore_index =  True)
    data_ssg.to_excel("data_ssg.xlsx", encoding ="cp949")
    
    return data_ssg

In [5]:
#실행코드
order_list = []
order_items = input("상품명: ")
order_list.append(order_items)
while order_items != "":
    order_items = input("")
    order_list.append(order_items)

ssglist_fin = ssg_shopping_list(order_list)
shoppinglist_ssg(ssglist_fin)

상품명: 사과
치즈
라면
오뚜기 케챱



,상품명,원가격,할인율,최종가격,비고
0,SSG.Fresh [프리미엄] 고당도사과 1.2kg 내외 (봉),13800,,13800,
1,[해피카우] 크림치즈 네추럴 150g,3190,,3190,
2,[농심] 올리브 짜파게티 (140gx5입),3280,,3280,
3,[오뚜기] 토마토케찹 300g,1380,,1380,
4,**합계**,21650,0,21650,
